Sentiment analysis of tweets regarding NBA games.

In [1]:
import pandas as pd
import datetime as datetime
from nltk.sentiment import SentimentIntensityAnalyzer
import numpy as np
import matplotlib.pyplot as plt

Done importing.

In [2]:
tweets_df = pd.read_csv('Datasets\mavs_games_tweets_merged.csv')
tweets_df.head()

,Unnamed: 0,Datetime,tweet_id,username,text,verified,retweet_count,like_count,Date,Opposing Team,Win or Loss,Score,Record,Leading Scorer-Winner,Leading Scorer Points,Leading Scorer Loss,Leading Scorer Points.1
0,0,2022-02-09 23:58:50+00:00,1491562265316773888,TheMontyShow,I told Mavs Bulls likeliest destination Collin...,False,0,1,2022-02-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2022-02-09 23:58:30+00:00,1491562180872974336,neilbobnuss,Your first counter offer gets laughed room haha,False,0,1,2022-02-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2022-02-09 23:58:30+00:00,1491562180281577480,tac234,"Trade Hield , Nurkic , know players actually h...",False,0,1,2022-02-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,2022-02-09 23:58:25+00:00,1491562158374723585,VeritasetfidePF,"Ja hj , ninguém sabe quem é . So aqui na rede ...",False,0,4,2022-02-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,2022-02-09 23:58:12+00:00,1491562107258687490,VCPHoops,"Mavs Post Game Interview : Jason Kidd , Trey B...",False,0,0,2022-02-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
